## Cost calculator for serving data from a Databricks SQl Warehouse (serverless) to the insight automation app.

It is expected that when a report is created several concurrent api requests will be made to the Warehouse, testing has shown that even the smallest available warehouse can handle 1000 concurrent requests without issue and serve the query results rapidly.

The primary driver of costs here is the compute uptime, which is a function of the time distribution of requests, the computes hourly DBU rate, and it's autotermination settings.

If the compute is terminated an incoming request will cause it to start up, once started it will serve any requests made, if no requests are made for the duration of the autotermination wait period then the cluster will terminate. The autotermination period starts from the completion of the last piece work assigned to the compute.

As as example suppose the autotermination is set to 10 minutes, then we can have the following two scenarios:

    17:00:00 Request received.
    17:00:01 Compute started
    17:00:05 Request completed.
    17:10:05 Compute terminated.

    17:12:00 Request received.
    17:12:01 Compute started
    17:12:05 Request completed.
    17:22:05 Compute terminated.

Total Uptime 20 minutes and 8 seconds to serve two requests.

    17:00:00 Request received.
    
    17:00:01 Compute started
    17:00:05 Request completed.

    17:07:00 Request received.
    17:07:04 Request completed.
    17:17:04 Compute terminated.

Total Uptime 17 minutes and 3 seconds to serve two requests.

This is relevant because clustered requests are more cost efficient, and so when calculating the cost of serving data to the reports generated we need to consider clustered requests.

In [40]:
import pandas as pd

In [41]:
# Cost variables
# These are variables which directly affect the compute cost of providing data to the insight automation app.
# Given that even the smallest serverless warehouse can happily handle 1000 + concurrent requests and server them all with 60 seconds,
# the primary cost driver is the uptime of the warehouse.



cluster_auto_termination_minutes = 10
cluster_hourly_dbu_cost = 6
dbu_cost = 1.25

In [42]:


data = pd.read_csv('data/report_intervals.csv')
df = pd.DataFrame(data)
df["CreatedOn"] = pd.to_datetime(df["CreatedOn"])
df["ProjectedComputeShutdown"] = df["CreatedOn"] + pd.to_timedelta(cluster_auto_termination_minutes, unit='m')
df = df.sort_values("CreatedOn")

In [43]:
intervals = []
current_interval = None
for _, row in df.iterrows():
    if current_interval is None:
        current_interval = {"start": row["CreatedOn"], "end": row["ProjectedComputeShutdown"]}
    else:
        if row["CreatedOn"] <= current_interval["end"]:
            current_interval["end"] = max(current_interval["end"], row["ProjectedComputeShutdown"])
        else:
            intervals.append(current_interval)
            current_interval = {"start": row["CreatedOn"], "end": row["ProjectedComputeShutdown"]}

if current_interval is not None:
    intervals.append(current_interval)

In [48]:
count_of_reports = len(df)
intervals_df = pd.DataFrame(intervals)
intervals_df["cluster_uptime_hours"] = (intervals_df["end"] - intervals_df["start"]).dt.total_seconds() / 3600
intervals_df["cluster_uptime_minutes"] = (intervals_df["end"] - intervals_df["start"]).dt.total_seconds() / 60


agg_df = intervals_df.agg({"cluster_uptime_hours": ["sum", "mean", "max", "min"],"cluster_uptime_minutes": ["sum", "mean", "max", "min"]})
agg_df["dbus_consumed"] = agg_df["cluster_uptime_hours"] * cluster_hourly_dbu_cost
agg_df["sterling_cost"] = agg_df["dbus_consumed"] * dbu_cost

print(
    f"""Statistics & cost for the {count_of_reports} reports created in 2024 had we used a serverless warehouse, 
    with a cluster auto-termination of {cluster_auto_termination_minutes} minutes, consuming {cluster_hourly_dbu_cost} an hour with a DBU cost of £{dbu_cost}."""
    )
print("#"*20, "stats and cost", "#"*20)
print(agg_df)


Statistics & cost for the 2415 reports created in 2024 had we used a serverless warehouse, 
    with a cluster auto-termination of 10 minutes, consuming 6 an hour with a DBU cost of £1.25.
#################### stats and cost ####################
      cluster_uptime_hours  cluster_uptime_minutes  dbus_consumed  \
sum             318.577344            19114.640661    1911.464066   
mean              0.197018               11.821052       1.182105   
max               0.722052               43.323096       4.332310   
min               0.166667               10.000000       1.000000   

      sterling_cost  
sum     2389.330083  
mean       1.477631  
max        5.415387  
min        1.250000  
